# RR class

In [1]:
import pandas as pd
import random
from uszipcode import SearchEngine
import numpy as np
import urllib.parse


class RestaurantRecommender:
    def __init__(self):
        self.business_df = pd.read_csv("tip_business.csv")
        self.tips_df = pd.read_csv("tip_text.csv")
        self.keywords = ["pizza", "sushi", "burger", "tacos", "ramen", "steak", "fried chicken", "pasta", "wings"]
        self.custom_pool = []

    def get_nearby_zipcodes(self, zipcode, radius):
        search = SearchEngine()
        zipcodes = search.by_zipcode(zipcode)

        if not zipcodes:
            print(f"Zip code '{zipcode}' not found.")
            return []

        lat, lng = zipcodes.lat, zipcodes.lng
        nearby_zipcodes = search.by_coordinates(lat, lng, radius=radius, returns=0)
        return [zipcode.zipcode for zipcode in nearby_zipcodes]

    def filter_by_zipcode(self, zipcode, radius):
        nearby_zipcodes = self.get_nearby_zipcodes(zipcode, radius)
        if not nearby_zipcodes:
            return []

        nearby_filtered_businesses = self.business_df[self.business_df["postal_code"].isin(nearby_zipcodes)]
        return nearby_filtered_businesses

    def filter_by_keyword(self, keyword, df):
        keyword_filtered_tips = self.tips_df[self.tips_df["text"].str.contains(keyword, case=False)]
        keyword_filtered_businesses = df[df["business_id"].isin(keyword_filtered_tips["business_id"])]
        return keyword_filtered_businesses, keyword_filtered_tips

    def compute_custom_score(self, row):
        return np.log(row["review_count"] + 1) * row["stars"] * row["tips_count"]

    def generate_google_maps_url(self, name, address):
        base_url = "https://www.google.com/maps/search/?api=1&query="
        query = f"{name} {address}"
        url_encoded_query = urllib.parse.quote(query)
        return base_url + url_encoded_query

    def recommender(self, keyword, zipcode, radius, n=3):
        nearby_filtered_businesses = self.filter_by_zipcode(zipcode, radius)
        filtered_by_keyword, keyword_filtered_tips = self.filter_by_keyword(keyword, nearby_filtered_businesses)

        filtered_by_keyword = filtered_by_keyword.copy()
        
        if filtered_by_keyword.empty:
            print("No restaurant found. Try increasing the search radius or using a different keyword.")
            return []
        filtered_by_keyword["tips_count"] = filtered_by_keyword["business_id"].apply(lambda x: len(keyword_filtered_tips[keyword_filtered_tips["business_id"] == x]))
        filtered_by_keyword["custom_score"] = filtered_by_keyword.apply(self.compute_custom_score, axis=1)

        top_n_restaurants = filtered_by_keyword.nlargest(n, "custom_score")

        recommendations = []
        for index, restaurant in top_n_restaurants.iterrows():
            stars = int(restaurant["stars"])
            half_stars = (restaurant["stars"] - stars) == 0.5
            star_str = "★" * stars
            if half_stars:
                star_str += "☆"

            tips_with_keyword = keyword_filtered_tips[keyword_filtered_tips["business_id"] == restaurant["business_id"]]
            tips_with_keyword_count = len(tips_with_keyword)
            random_tips_with_keyword = tips_with_keyword.sample(min(3, tips_with_keyword_count))["text"].tolist()

            location = f"{restaurant['address']}, {restaurant['city']}, {restaurant['state']} {restaurant['postal_code']}"
            google_maps_url = self.generate_google_maps_url(restaurant["name"], location)

            recommendations.append({
                "name": restaurant['name'],
                "stars": star_str,
                "keyword": keyword,
                "tips": random_tips_with_keyword,
                "tips_count": tips_with_keyword_count,
                "location": location,
                "custom_score": restaurant["custom_score"],
                "google_maps_url": google_maps_url
            })

        return recommendations
    
    def draw(self, zipcode, radius, keywords=None):
        if not keywords:
            keywords = self.keywords

        random_keyword = random.choice(keywords)
        recommendations = self.recommender(random_keyword, zipcode, radius)

        if recommendations:
            random_recommendation = random.choice(recommendations)
            return random_recommendation
        else:
            return None


/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# recommender = RestaurantRecommender()

# recommender.recommender(keyword="chicken", zipcode="100110", radius=5, n=3)

# Interface 

In [3]:
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import textwrap

recommender = RestaurantRecommender()

def on_search_click(button):
    clear_output()
    display(search_box)

    
    results = recommender.recommender(keyword_input.value, zipcode_input.value, int(radius_input.value))

    if not results:
        return

    for index, result in enumerate(results, 1):
        display(HTML(f"<div style='text-align:center; margin-top:10px;'><a href='{result['google_maps_url']}' target='_blank' style='font-size: 20px; font-weight: bold; text-decoration: none;'>{result['name']} ({result['stars']})</a></div>"))
        display(HTML(f"<div style='text-align:center;'>{result['location']}</div>"))
        display(HTML(f"<div style='text-align:center; font-size: 18px;'>{result['tips_count']} tips mentioned {result['keyword']}</div>"))
        for tip in result['tips']:
            wrapped_tip = textwrap.fill(tip, width=80)
            wrapped_tip = wrapped_tip.replace('\n', '<br>')
            display(HTML(f"<div style='text-align:center;'><i>- \"{wrapped_tip}\"</i></div>"))

            
search_title = widgets.HTML(value="<h3>Restaurant Recommender</h3>")
keyword_input = widgets.Text(
    value = "wings",
    placeholder='e.g., burger, sushi',
    description='Cuisine:',
    disabled=False
)

zipcode_input = widgets.Text(description="Zip Code:", value= "19108")

radius_input = widgets.FloatSlider(
    value=10,
    min=5,
    max=50.0,
    step=5,
    description='Mile Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f'
)

search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='info',
    tooltip='Search for restaurants',
    icon='search'
)
search_button.on_click(on_search_click)

input_row = HBox([keyword_input, zipcode_input, radius_input])
search_row = HBox([search_button], layout=widgets.Layout(justify_content='center', margin='10px 0 0 0'))

search_box = VBox([search_title, input_row, search_row], layout=widgets.Layout(display='flex', flex_flow='column', align_items='center'))

# define the global variable
costomized_pool = None 

def on_submit_custom_pool(btn):
    global costomized_pool
    custom_pool_output.clear_output()
    selected_food = [btn.description for btn in food_buttons if btn.value]
    
    # Split custom input by semicolon and strip whitespace
    if custom_pool_input.value:
        custom_food = [food.strip() for food in custom_pool_input.value.split(';')]
        selected_food.extend(custom_food)
    costomized_pool = selected_food
    with custom_pool_output:
        display(HTML(f"<div style='text-align:center; '>Your  Pool</div>"))
        display(HTML(', '.join([f"<span style='color:#C70039;'>{food}</span>" for food in selected_food])))


custom_pool_title = widgets.HTML(value="<h3 style='text-align:center;'>Dine Dice<br>Step 1: Customize Your Pool</h3>")

food_options = ["pizza", "sushi", "burger", "tacos", "ramen", "steak", "fried chicken", "pasta", "wings", "sashimi", "ribeye", "seafood"]

# Add some style to the ToggleButton
toggle_button_style = {'button_width': '100px', 'button_height': '35px', 'button_margin': '5px'}

food_buttons = [widgets.ToggleButton(description=food, layout=widgets.Layout(width=toggle_button_style['button_width'], height=toggle_button_style['button_height'], margin=toggle_button_style['button_margin'])) for food in food_options]

custom_pool_input = widgets.Text(placeholder="Or enter by hand, separated by ';'")

custom_pool_submit_button = widgets.Button(description="Submit", button_style='info')
custom_pool_submit_button.on_click(on_submit_custom_pool)

custom_pool_output = widgets.Output()

buttons_grid = widgets.GridBox(food_buttons, layout=widgets.Layout(grid_template_columns="repeat(4, 100px)"))
custom_pool_box = VBox([custom_pool_title, buttons_grid, custom_pool_input, custom_pool_submit_button, custom_pool_output], layout=widgets.Layout(display='flex', flex_flow='column', align_items='center'))


def on_draw_button_click(btn):
    output.clear_output()

    result = recommender.draw(draw_zipcode_input.value, draw_radius_input.value, costomized_pool)
    if result:
        with output:
            display(HTML(f"<div style='text-align:center; font-size: 21px; margin-top:10px;'>Let's try <span style='color:#C70039;'>{result['keyword']}</span></div>"))
            display(HTML(f"<div style='text-align:center; margin-top:10px;'><a href='{result['google_maps_url']}' target='_blank' style='font-size: 20px; font-weight: bold; text-decoration: none;'>{result['name']} ({result['stars']})</a></div>"))
            display(HTML(f"<div style='text-align:center;'>{result['location']}</div>"))
            display(HTML(f"<div style='text-align:center; font-size: 18px;'>{result['tips_count']} tips mentioned {result['keyword']}</div>"))
            for tip in result['tips']:
                wrapped_tip = textwrap.fill(tip, width=80)
                wrapped_tip = wrapped_tip.replace('\n', '<br>')
                display(HTML(f"<div style='text-align:center;'><i>- \"{wrapped_tip}\"</i></div>"))

    else:
        output.clear_output()
        with output:

            display(HTML("<div style='text-align:center; color:red;'>No recommendation found. Please try again.</div>"))

title = widgets.HTML(value="<h3>Step 2: Draw a Lot!</h3>")

draw_zipcode_input = widgets.Text(description="Zip Code:", value="19107")
draw_radius_input = widgets.FloatSlider(
    value=10,
    min=5,
    max=50.0,
    step=5,
    description='Mile Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f'
)

submit_button = widgets.Button(description="Draw", button_style='info')
submit_button.on_click(on_draw_button_click)

output = widgets.Output()

input_row = HBox([draw_zipcode_input, draw_radius_input])
submit_row = HBox([submit_button], layout=widgets.Layout(justify_content='center'))

input_box = VBox([input_row, submit_row], layout=widgets.Layout(display='flex', flex_flow='column', align_items='center'))
main_box = VBox([title, input_box, output], layout=widgets.Layout(display='flex', flex_flow='column', align_items='center'))

draw_box = VBox([custom_pool_box, main_box], layout=widgets.Layout(display='flex', flex_flow='column', align_items='center'))

In [4]:
def show_draw_box(button):
    with main_output:
        clear_output()
        display(draw_box)

def show_search_box(button):
    with main_output:
        clear_output()
        display(search_box)

draw_link = widgets.Button(
    description='Not sure what to eat?',
    layout=widgets.Layout(width='auto'),
    button_style='',  # Remove the default button style
    tooltip='Click to switch to the draw box',
    style={'button_color': 'white', 'font_weight': 'normal', 'text_decoration': 'underline'}
)
draw_link.on_click(show_draw_box)

search_link = widgets.Button(
    description='Back to search',
    layout=widgets.Layout(width='auto'),
    button_style='',  # Remove the default button style
    tooltip='Click to switch to the search box',
    style={'button_color': 'white', 'font_weight': 'normal', 'text_decoration': 'underline'}
)
search_link.on_click(show_search_box)

# Add the draw_link to the search_box input section
search_box.children += (draw_link,)

# Add the search_link to the draw_box input section
draw_box.children += (search_link,)

main_output = widgets.Output()

with main_output:
    display(search_box)

display(main_output)

Output()